In [4]:
%matplotlib inline
import os
import pandas as pd
import torch
import torchvision
from d2l import torch as d2l

In [5]:
d2l.DATA_HUB['banana-detection'] = (
    d2l.DATA_URL + 'banana-detection.zip',
    '5de26c8fce5ccdea9f91267273464dc968d20d72')

In [6]:
# 读取香蕉检测数据集
def read_data_bananas(is_train = True):
    """读取香蕉检测数据集中的图像和标签"""
    data_dir = d2l.download_extract('banana-detection')
    csv_fname = os.path.join(data_dir,
                             'bananas_train' if is_train else 'bananas_val',
                             'label.csv')
    csv_data = pd.read_csv(csv_fname)
    csv_data = csv_data.set_index('img_name')
    images,targets = [],[]
    for img_name,target in csv_data.iterrows():
        images.append(
            torchvision.io.read_image(
                os.path.join(data_dir,
                             'bananas_train' if is_train else 'bananas_val',
                             'images',
                             f'{img_name}')))
        target.append(list(target))
    return images,torch.tensor(target).unsqueeze(1)/256

In [7]:
# 创建一个自定义的Dataset实例
class BananasDataset(torch.utils.data.Dataset):
    """"一个用于加载香蕉检测数据集的自定义数据集"""
    def __init__(self,is_train):
        self.features,self.labels = read_data_bananas(is_train)
        print('read'+str(len(self.features))+
              (f'trianing examples' if is_train else f'validation examples'))
    def __getitem__(self,idx):
        return (self.features[idx].float(),self.labels[idx])
    def __len__(self):
        return len(self.features)